# WGAN-GP

## 1. Data Load

In [ ]:
import pandas as pd
import numpy as np
import pickle
import sqlite3

dataset = 
display(dataset)

In [ ]:
train_X =

print(train_X.shape)


## 6. GPU Diet

In [ ]:
#######################################################################################
import tensorflow as tf
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

from tensorflow.config.experimental import list_physical_devices, set_memory_growth
gpus = list_physical_devices('GPU')
display(gpus)
if gpus:
  try:
    set_memory_growth(gpus[0], True)
  except RuntimeError as e:
    print(e)
    
#######################################################################################

## 7. Critic

In [ ]:
#critic
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Conv1D, LeakyReLU, Flatten, Dropout, Lambda, BatchNormalization, Input, MaxPooling1D, Add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.backend import clear_session
from tensorflow.keras import regularizers

clear_session()

# YOUR PARAMTERS
#######################################################################################

np.random.seed(1500)
tf.random.set_seed(1500)

kernel_size = 7
pool_size = train_X.shape[1] - kernel_size + 1

#######################################################################################

def convolution_block(x, filters, size, strides):
    
    bn = BatchNormalization()(x)
    l_relu = LeakyReLU()(bn)
    conv1d = Conv1D(filters=filters, kernel_size=size, strides=strides, kernel_regularizer=regularizers.l1(1e-5),padding='same')(l_relu)
    
    return conv1d
    
def create_critic(input_dim):
    
    # MODEL
    #######################################################################################
    
    inp = Input(shape=input_dim)
    conv1 = Conv1D(filters=32, kernel_size=7, strides=1, kernel_regularizer=regularizers.l1(1e-5),padding='same')(inp)
    convb1 = convolution_block(conv1, filters=64, size=7, strides=1)
    convb2 = convolution_block(convb1, filters=64, size=7, strides=1)
    convb2 = convolution_block(convb2, filters=32, size=7, strides=1)
    add1 = Add()([conv1, convb2])
    mp1 = MaxPooling1D(pool_size=input_dim[0])(add1)
    
    flatten = Flatten()(mp1)
    
    fc1 = Dense(32, kernel_regularizer=regularizers.l1(1e-5))(flatten)
    l_relu1 = LeakyReLU()(fc1)
    dropout1 = Dropout(rate=0.25)(l_relu1)
    outp = Dense(1, kernel_regularizer=regularizers.l1(1e-5))(dropout1)
    
    model = Model(inp, outp)

    #######################################################################################
    
    model.summary()
    
    return model

critic = create_critic(train_X.shape[1:])

## 8. Generator

In [ ]:
#generator
from tensorflow.keras.layers import Conv1DTranspose, Reshape, Activation, UpSampling1D, ZeroPadding1D, Input, BatchNormalization, ReLU
from tensorflow.keras import regularizers

z_dim = 100

# YOUR PARAMTERS
#######################################################################################

design_len = 30

#######################################################################################

def create_generator(z_dim):
    model = Sequential(name="generator")
    # MODEL
    #######################################################################################
    model.add(Input(z_dim))
    model.add(Dense(64, kernel_regularizer=regularizers.l1(1e-5)))
#     model.add(LeakyReLU())
    model.add(Dense(32, kernel_regularizer=regularizers.l1(1e-5)))
    model.add(Reshape((1, 32)))
    model.add(Conv1DTranspose(filters=4, kernel_size=design_len, strides=1, kernel_regularizer=regularizers.l1(1e-5)))

    model.add(Activation("softmax"))
#     model.add(Lambda(gumbel_softmax)
    
#     model.add(Lambda(lambda x: x-.25))
#     model.add(ZeroPadding1D(padding=(9, 9)))
#     model.add(Lambda(lambda x: x+.25))

    #######################################################################################
    
    model.summary()

    return model

generator = create_generator(z_dim)

## 8. Optimizers

In [ ]:
from tensorflow.keras.optimizers import Adam

critic_opt = Adam(learning_rate = 0.0001, beta_1=0, beta_2=0.9)
generator_opt = Adam(learning_rate = 0.0001, beta_1=0, beta_2=0.9)

## 9. Train Critic

In [ ]:
def train_critic(seq, sample_weights):
    batch_size = len(seq)
    z = np.random.normal(0, 1, (batch_size, 100))
    gp_lambda = 10
    
    with tf.GradientTape() as critic_tape:
        critic.training = True
        gen_seq = generator(z)
        output_real = critic(seq)
        output_fake = critic(gen_seq)
        
        #wgan loss
        loss = K.mean(output_fake) - K.mean(output_real)
        
        e = tf.random.normal([batch_size, 1, 1], 0.0, 1.0)
        x_hat = e * seq - (1 - e) * gen_seq
    
        
        with tf.GradientTape() as t:
            t.watch(x_hat)
            c_hat = critic(x_hat)
            
        gradients = t.gradient(c_hat, [x_hat])
        l2_norm = K.sqrt(K.sum(K.square(gradients), axis=[2,3]))
        l2_norm = K.squeeze(l2_norm, axis=0)
        gradient_penalty = K.sum(K.square((l2_norm-1.)), axis=[0])
        
        loss += gp_lambda*gradient_penalty
        
    g_critic = critic_tape.gradient(loss, critic.trainable_variables)
    critic_opt.apply_gradients(zip(g_critic, critic.trainable_variables))
    
    return K.sum(loss)

## 10. Train Generator Function

In [ ]:
def train_generator(batch_size):
    z = np.random.normal(0, 1, (batch_size, 100))
    
    with tf.GradientTape() as generator_tape:
        generator.training = True
        gen_seq = generator(z)
        output_fake = critic(gen_seq)
        
        loss = - K.mean(output_fake)
        
    g_generator = generator_tape.gradient(loss, generator.trainable_variables)
    generator_opt.apply_gradients(zip(g_generator, generator.trainable_variables))
    
    return K.sum(loss)

## 13. Train Model

In [ ]:
import random
from tqdm import tqdm

batch_size = 64

def train(dataset, importance, epoch):
    for epoch in range(epoch):
        g_loss = []
        c_loss = []
        
        shuf = np.array([i for i in range(train_X.shape[0])])
        random.shuffle(shuf)

        dataset = dataset[shuf]
        
        for batch in tqdm(range(dataset.shape[0] // batch_size)):
            real_X = dataset[batch * batch_size:(batch+1)*batch_size]
            
            loss_c = 0
            for i in range(5):
                loss_c += train_critic(real_X, 1)
            loss_g = train_generator(batch_size)
        
            g_loss.append(loss_g)
            c_loss.append(loss_c / 5)
        
        # loss & plot
        print ('epoch {}'.format(epoch + 1))
        print (epoch+1, ":", "g_loss: {}".format(sum(g_loss)/len(g_loss)))
        print (epoch+1, ":", "c_loss: {}".format(sum(c_loss)/len(c_loss)))
         

        show_plot()
        write_gen_fasta(epoch)
        print("="*80)
    
    retun 

In [ ]:
train(train_X, importance, 100)
# train(train_X, 1, 100)